In [0]:
from pyspark.sql.functions import current_timestamp, when, col
import requests, time
from pyspark.sql.types import StringType
import json
from datetime import datetime, timedelta

catalog_name = "dev_ridb"
schema_name = "bronze"
table_name = "facilities"
api_key = dbutils.secrets.get(scope="ridb_secrets", key="apikey")
api_url = "https://ridb.recreation.gov/api/v1/facilities"
page_size = 50
delay_seconds = 0.025

spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_name}")
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_name}")
spark.sql(f"USE SCHEMA {schema_name}")

def get_page(offset):
    params = {"limit": page_size, "offset": offset}
    headers = {"accept": "application/json", "apikey": api_key}
    response = requests.get(api_url, headers=headers, params=params)
    if response.status_code != 200:
        raise Exception(f"API error: {response.status_code} - {response.text}")
    return response.json().get("RECDATA", [])

# Fetch loop
all_records = []
offset = 0
while True:
    data = get_page(offset)
    print(f"Fetched offset {offset} with {len(data)} records")
    if not data:
        break
    all_records.extend(data)
    offset += page_size  # increment by 50
    time.sleep(delay_seconds)

print(f"Total records fetched: {len(all_records)}")

# Serialize any nested or mixed-type fields to JSON text
for record in all_records:
    for k, v in record.items():
        if isinstance(v, (dict, list)):
            record[k] = json.dumps(v)
        elif isinstance(v, (int, float)):   # normalize numeric types to string
            record[k] = str(v)


df = spark.createDataFrame(all_records)

for c in df.columns:
    df = df.withColumn(c, when(col(c) == "", None).otherwise(col(c)))

df = df.withColumn("ingested_at", current_timestamp())

table_path = f"{catalog_name}.{schema_name}.{table_name}"

if not spark.catalog.tableExists(table_path):
    df.write.format("delta") \
        .option("delta.enableChangeDataFeed", "true") \
        .saveAsTable(table_path)
else:
    df.createOrReplaceTempView("staging_facilities")
    spark.sql(f"""
        MERGE INTO {table_path} t
        USING staging_facilities s
        ON t.FacilityID = s.FacilityID AND t.LastUpdatedDate = s.LastUpdatedDate
        WHEN NOT MATCHED THEN INSERT *
    """)

In [0]:
display(df.limit(100))

In [0]:
%sql
SELECT *
FROM dev_ridb.bronze.facilities
WHERE facilityname IN (
  SELECT facilityname
  FROM dev_ridb.bronze.facilities
  GROUP BY facilityname
  HAVING COUNT(*) > 1
  )
  ORDER BY facilityname

In [0]:
%sql
SELECT * FROM dev_ridb.bronze.facilities ORDER BY rand() limit 5000